# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/23/24 21:04:44] INFO     Found credentials from IAM Role:                                   ]8;id=445228;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=69813;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

[12/23/24 21:04:49] INFO     Found credentials from IAM Role:                                   ]8;id=341234;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=582694;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

arn:aws:iam::796973520885:role/service-role/AmazonSageMaker-ExecutionRole-20241219T112240


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://urban-object-detection-project/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [6]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
[+] Building 0.0s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.1s (2/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
[+] Building 0.3s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                  

[+] Building 1.1s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  1.0s
 => => # Get:13 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 k
 => => # B]                                                                    
 => => # Get:14 http://archive.ubuntu.co

[+] Building 2.0s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  1.9s
 => => # Get:22 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packa
 => => # ges [4639 kB]                                                         
 => => # Get:23 http://archive.ubuntu.co

[+] Building 2.8s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  2.7s
 => => # Get:23 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Pac
 => => # kages [55.2 kB]                                                       
 => => # Get:24 http://archive.ubuntu.co

[+] Building 3.7s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  3.6s
 => => # Get:23 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Pac
 => => # kages [55.2 kB]                                                       
 => => # Get:24 http://archive.ubuntu.co

...

[+] Building 93.5s (15/16)                                       docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  22.5s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.2s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.2s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 94.3s (15/16)                                       docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  22.5s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.2s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.2s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 95.0s (15/16)                                       docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  22.5s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.2s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.2s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 95.8s (15/16)                                       docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  22.5s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.2s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.2s
 => [ 5/11] WORKDIR /home/tensorflow/mod

Pushing image to ECR 796973520885.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20241223210503
The push refers to repository [796973520885.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection]

7f44dbff: Preparing 
85b41407: Preparing 
b7848892: Preparing 
7086aae6: Preparing 
1e17d7d4: Preparing 
b9853478: Preparing 
bf18a086: Preparing 
6ba112ea: Preparing 
e407db4c: Preparing 
1d2dc4fb: Preparing 
a40e4dcd: Preparing 
b5695a98: Preparing 
f0d116f4: Preparing 
2813a979: Preparing 
6e868aa5: Preparing 
136c7d36: Preparing 
891e0e76: Preparing 
ed848ac5: Preparing 
18b47754: Preparing 
91e05b94: Preparing 
e103257c: Preparing 
b25399cb: Preparing 
b667a965: Preparing 
6ad9c95e: Preparing 
b4e1ecd1: Preparing 
5c845fcf: Preparing 
f18a086: Waiting g 
7848892: Pushed   1.486GB/1.462GBPushing  45.79MB/95.28MBPushing  61.75MB/95.28MBPushing  124.1MB/1.462GBPushing  312.3MB/1.462GBPushing  485.4MB/1.462GBPushing  1.087GB/1.462GB20241223210503: digest: sha256:94952fea7989af6222c1d9cc7

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

796973520885.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20241223210503


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [8]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/ssd_resnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssd_resnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2024-12-23 21:08:04--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.207, 142.251.111.207, 142.251.163.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/tmp/ssd_resnet50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 15.2M 15s
    50K .......... .......... .......... .......... ..........  0% 25.4M 12s
   100K .......... .......... .......... .......... ..........  0% 30.5M 11s
   150K .......... .......... .......... .......... ..........  0% 92.4M 9s
   200K .......... .......... .......... .......... ..........  0% 93.6M 7s
 

  5000K .......... .......... .......... .......... ..........  2%  173M 5s
  5050K .......... .......... .......... .......... ..........  2%  287M 5s
  5100K .......... .......... .......... .......... ..........  2%  197M 5s
  5150K .......... .......... .......... .......... ..........  2%  215M 5s
  5200K .......... .......... .......... .......... ..........  2%  247M 5s
  5250K .......... .......... .......... .......... ..........  2%  282M 5s
  5300K .......... .......... .......... .......... ..........  2%  209M 5s
  5350K .......... .......... .......... .......... ..........  2%  293M 5s
  5400K .......... .......... .......... .......... ..........  2%  183M 5s
  5450K .......... .......... .......... .......... ..........  2%  308M 5s
  5500K .......... .......... .......... .......... ..........  2%  214M 5s
  5550K .......... .......... .......... .......... ..........  2%  248M 5s
  5600K .......... .......... .......... .......... ..........  2%  173M 5s
  5650K ....

 10400K .......... .......... .......... .......... ..........  4%  243M 3s
 10450K .......... .......... .......... .......... ..........  4%  255M 3s
 10500K .......... .......... .......... .......... ..........  4%  306M 3s
 10550K .......... .......... .......... .......... ..........  4%  259M 3s
 10600K .......... .......... .......... .......... ..........  4%  260M 3s
 10650K .......... .......... .......... .......... ..........  4%  314M 3s
 10700K .......... .......... .......... .......... ..........  4%  295M 3s
 10750K .......... .......... .......... .......... ..........  4%  316M 3s
 10800K .......... .......... .......... .......... ..........  4%  238M 3s
 10850K .......... .......... .......... .......... ..........  4%  309M 3s
 10900K .......... .......... .......... .......... ..........  4%  299M 3s
 10950K .......... .......... .......... .......... ..........  4%  305M 3s
 11000K .......... .......... .......... .......... ..........  4%  194M 3s
 11050K ....

 15800K .......... .......... .......... .......... ..........  6%  222M 2s
 15850K .......... .......... .......... .......... ..........  6%  212M 2s
 15900K .......... .......... .......... .......... ..........  6%  209M 2s
 15950K .......... .......... .......... .......... ..........  6%  292M 2s
 16000K .......... .......... .......... .......... ..........  6%  191M 2s
 16050K .......... .......... .......... .......... ..........  6%  228M 2s
 16100K .......... .......... .......... .......... ..........  6%  258M 2s
 16150K .......... .......... .......... .......... ..........  6%  197M 2s
 16200K .......... .......... .......... .......... ..........  6%  178M 2s
 16250K .......... .......... .......... .......... ..........  6%  278M 2s
 16300K .......... .......... .......... .......... ..........  6%  279M 2s
 16350K .......... .......... .......... .......... ..........  6%  210M 2s
 16400K .......... .......... .......... .......... ..........  6%  167M 2s
 16450K ....

 21200K .......... .......... .......... .......... ..........  8%  176M 2s
 21250K .......... .......... .......... .......... ..........  8%  222M 2s
 21300K .......... .......... .......... .......... ..........  8%  306M 2s
 21350K .......... .......... .......... .......... ..........  8%  205M 2s
 21400K .......... .......... .......... .......... ..........  8%  253M 2s
 21450K .......... .......... .......... .......... ..........  8%  271M 2s
 21500K .......... .......... .......... .......... ..........  9%  211M 2s
 21550K .......... .......... .......... .......... ..........  9%  267M 2s
 21600K .......... .......... .......... .......... ..........  9%  176M 2s
 21650K .......... .......... .......... .......... ..........  9%  202M 2s
 21700K .......... .......... .......... .......... ..........  9%  312M 2s
 21750K .......... .......... .......... .......... ..........  9%  184M 2s
 21800K .......... .......... .......... .......... ..........  9%  227M 2s
 21850K ....

...

222200K .......... .......... .......... .......... .......... 92%  382M 0s
222250K .......... .......... .......... .......... .......... 92%  299M 0s
222300K .......... .......... .......... .......... .......... 93%  408M 0s
222350K .......... .......... .......... .......... .......... 93%  279M 0s
222400K .......... .......... .......... .......... .......... 93%  348M 0s
222450K .......... .......... .......... .......... .......... 93%  328M 0s
222500K .......... .......... .......... .......... .......... 93%  323M 0s
222550K .......... .......... .......... .......... .......... 93%  274M 0s
222600K .......... .......... .......... .......... .......... 93%  405M 0s
222650K .......... .......... .......... .......... .......... 93%  379M 0s
222700K .......... .......... .......... .......... .......... 93%  310M 0s
222750K .......... .......... .......... .......... .......... 93%  332M 0s
222800K .......... .......... .......... .......... .......... 93%  412M 0s
222850K ....

227600K .......... .......... .......... .......... .......... 95%  323M 0s
227650K .......... .......... .......... .......... .......... 95%  374M 0s
227700K .......... .......... .......... .......... .......... 95%  386M 0s
227750K .......... .......... .......... .......... .......... 95%  309M 0s
227800K .......... .......... .......... .......... .......... 95%  406M 0s
227850K .......... .......... .......... .......... .......... 95%  392M 0s
227900K .......... .......... .......... .......... .......... 95%  423M 0s
227950K .......... .......... .......... .......... .......... 95%  278M 0s
228000K .......... .......... .......... .......... .......... 95%  331M 0s
228050K .......... .......... .......... .......... .......... 95%  385M 0s
228100K .......... .......... .......... .......... .......... 95%  350M 0s
228150K .......... .......... .......... .......... .......... 95%  320M 0s
228200K .......... .......... .......... .......... .......... 95%  363M 0s
228250K ....

233000K .......... .......... .......... .......... .......... 97%  329M 0s
233050K .......... .......... .......... .......... .......... 97%  353M 0s
233100K .......... .......... .......... .......... .......... 97%  264M 0s
233150K .......... .......... .......... .......... .......... 97%  329M 0s
233200K .......... .......... .......... .......... .......... 97%  334M 0s
233250K .......... .......... .......... .......... .......... 97%  353M 0s
233300K .......... .......... .......... .......... .......... 97%  298M 0s
233350K .......... .......... .......... .......... .......... 97%  318M 0s
233400K .......... .......... .......... .......... .......... 97%  401M 0s
233450K .......... .......... .......... .......... .......... 97%  329M 0s
233500K .......... .......... .......... .......... .......... 97%  340M 0s
233550K .......... .......... .......... .......... .......... 97%  364M 0s
233600K .......... .......... .......... .......... .......... 97%  358M 0s
233650K ....

238400K .......... .......... .......... .......... .......... 99%  229M 0s
238450K .......... .......... .......... .......... .......... 99%  338M 0s
238500K .......... .......... .......... .......... .......... 99%  194M 0s
238550K .......... .......... .......... .......... .......... 99%  240M 0s
238600K .......... .......... .......... .......... .......... 99%  312M 0s
238650K .......... .......... .......... .......... .......... 99%  306M 0s
238700K .......... .......... .......... .......... .......... 99%  213M 0s
238750K .......... .......... .......... .......... .......... 99%  287M 0s
238800K .......... .......... .......... .......... .......... 99%  322M 0s
238850K .......... .......... .......... .......... .......... 99%  212M 0s
238900K .......... .......... .......... .......... .......... 99%  256M 0s
238950K .......... .......... .......... .......... .......... 99%  257M 0s
239000K .......... .......... .......... .......... .......... 99%  313M 0s
239050K ....

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [9]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "ssd_resnet50_v1.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

[12/23/24 21:09:07] INFO     Found credentials from IAM Role:                                   ]8;id=560775;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=197922;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=847293;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=389755;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=527128;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=217419;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[12/23/24 21:09:15] INFO     Creating training-job with name:                                       ]8;id=286981;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=915136;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tf2-object-detection-2024-12-23-21-09-07-716                                          

2024-12-23 21:09:16 Starting - Starting the training job...
2024-12-23 21:09:31 Starting - Preparing the instances for training...
2024-12-23 21:10:12 Downloading - Downloading the training image.........
2024-12-23 21:11:43 Training - Training image download completed. Training in progress...2024-12-23 21:11:57,900 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 21:11:57,937 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 21:11:57,974 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 21:11:57,990 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluster",
    "current_instance_gro

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W1223 21:12:10.811837 140393317267264 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W1223 21:12:13.824354 140393317267264 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W1223 21:12:16.084308 140393

INFO:tensorflow:Step 300 per-step time 0.391s
I1223 21:15:11.256620 140393317267264 model_lib_v2.py:705] Step 300 per-step time 0.391s
INFO:tensorflow:{'Loss/classification_loss': 0.5086679,
 'Loss/localization_loss': 0.03568064,
 'Loss/regularization_loss': 0.27978617,
 'Loss/total_loss': 0.8241347,
 'learning_rate': 0.01705}
I1223 21:15:11.256875 140393317267264 model_lib_v2.py:708] {'Loss/classification_loss': 0.5086679,
 'Loss/localization_loss': 0.03568064,
 'Loss/regularization_loss': 0.27978617,
 'Loss/total_loss': 0.8241347,
 'learning_rate': 0.01705}
INFO:tensorflow:Step 400 per-step time 0.392s
I1223 21:15:50.415159 140393317267264 model_lib_v2.py:705] Step 400 per-step time 0.392s
INFO:tensorflow:{'Loss/classification_loss': 0.6342275,
 'Loss/localization_loss': 0.033614144,
 'Loss/regularization_loss': 0.27960545,
 'Loss/total_loss': 0.9474471,
 'learning_rate': 0.0184}
I1223 21:15:50.415425 140393317267264 model_lib_v2.py:708] {'Loss/classification_loss': 0.6342275,
 'Loss

INFO:tensorflow:Step 1700 per-step time 0.392s
I1223 21:24:21.520295 140393317267264 model_lib_v2.py:705] Step 1700 per-step time 0.392s
INFO:tensorflow:{'Loss/classification_loss': 0.9203905,
 'Loss/localization_loss': 0.078173816,
 'Loss/regularization_loss': 4.332194,
 'Loss/total_loss': 5.330758,
 'learning_rate': 0.035949998}
I1223 21:24:21.520533 140393317267264 model_lib_v2.py:708] {'Loss/classification_loss': 0.9203905,
 'Loss/localization_loss': 0.078173816,
 'Loss/regularization_loss': 4.332194,
 'Loss/total_loss': 5.330758,
 'learning_rate': 0.035949998}
INFO:tensorflow:Step 1800 per-step time 0.391s
I1223 21:25:00.659597 140393317267264 model_lib_v2.py:705] Step 1800 per-step time 0.391s
INFO:tensorflow:{'Loss/classification_loss': 3.7881444,
 'Loss/localization_loss': 0.06505439,
 'Loss/regularization_loss': 4.2357836,
 'Loss/total_loss': 8.088983,
 'learning_rate': 0.037299998}
I1223 21:25:00.659859 140393317267264 model_lib_v2.py:708] {'Loss/classification_loss': 3.78814

I1223 21:26:50.214533 139648953096000 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use `tf.cast` instead.
W1223 21:26:54.709680 139648953096000 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Finished eval step 0
I1223 21:26:54.750044 139648953096000 model_lib_v2.py:966] Finished eval step 0
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well a

I1223 21:31:59.465401 140035990218560 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1223 21:32:02.113267 140035990218560 signature_serialization.py:148] Function `call_func` contains input name(s) resource with unsupported characters which will be renamed to weightsharedconvolutionalboxpredictor_classpredictiontower_conv2d_3_batchnorm_feature_4_fusedbatchnormv3_readvariableop_1_resource in the SavedModel.
I1223 21:32:03.213551 140035990218560 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
W1223 21:32:05.085186 140035990218560 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f5bfc05b610>, because it is not built.
I1223 21:32:15.587448 140035990218560 save.py:274] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_lo

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

## Visualize with Tensorboard
!pip install tensorboard

Visualize the tensor log on the terminal using  the tensorboard on the command prompt

tensor board --logdir="path to tensor log file"

We visualize the log of the tf records found in "logs/ssd_resnet50_v1/train/" for the training log and "logs/ssd_resnet50_v1/eval/" for the evaluation logs as seen in the output below
## Sample of Training Images 
![ssd_resnet50_v1_train_image](../figures/ssd_resnet50_v1/train/ssd_resnet50_v1_images_train.png)

## Training log
![ssd_resnet50_v2_train_log](../figures/ssd_resnet50_v1/train/ssd_resnet50_v1_train.png)


### Sample of evaluation images
![ssd_resnet50_v1_eval_image](../figures/ssd_resnet50_v1/eval/ssd_resnet50_v1_eval_images.png)

### Evaluation Log
![ssd_resnet50_v1_eval_log](../figures/ssd_resnet50_v1/eval/ssd_resnet50_v1_eval.png)


### Model Report

**Data Augmentation**
Data Augmentation done for this model as seen in `source_dir/ssd_resnet50_v1.config` is :
1. Random Horizontal Flip

2. Random scale and crop and pad to square

3. Random vertical flip

4. Random adjust brightness

5. Random adjust contrast

#### Training log Report  

**Training Losses at step=2000:**

1. Classification Loss : **0.8033**

2. Localization Loss : **0.04502**

3. Regularization Loss : **4.119**

4. Total Loss : **4.967**

**Learning rate** : **0.04**

**Steps per seconds** : **2.556**



### Evaluation Log Report


**Validation Losses at step = 2000 :**

1. Classification Loss : **0.2943**

2. Localization Loss : **0.02444**

3. Regularization Loss : **0.1432**

4. Total Loss : **0.4619**

**Detection_Boxes_Precision**

1. mAP = **9.2938 * 10 ^ -5**

2. mAP( large) = **1.4657 * 10 ^ -3**

3. mAP (medium) = **0**

4. mAP (small) = **0**

5. mAP @ 0.5IoU = **3.9512 * 10 ^ -4** 

6. mAP @ 0.75IoU = **9.568 * 10 ^ -6**
